![Neptune + TensorFlow](https://neptune.ai/wp-content/uploads/2023/09/tensorboard_tensorflow.svg)

# Using Neptune with TensorFlow

<a target="_blank" href="https://colab.research.google.com/github/neptune-ai/examples/blob/main/integrations-and-supported-tools/tensorflow/notebooks/Neptune_Tensorflow.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/>
</a><a target="_blank" href="https://github.com/neptune-ai/examples/blob/main/integrations-and-supported-tools/tensorflow/notebooks/Neptune_Tensorflow.ipynb">
  <img alt="Open in GitHub" src="https://img.shields.io/badge/Open_in_GitHub-blue?logo=github&labelColor=black">
</a><a target="_blank" href="https://app.neptune.ai/o/common/org/tensorflow-support/runs/details?viewId=97f6e558-ba33-4f88-9bc9-0e55347fb79a&detailsTab=dashboard&dashboardId=97f6ac04-2c4b-4d10-97da-cd3a51bbeec8&shortId=TFSUP-101"> 
  <img alt="Explore in Neptune" src="https://neptune.ai/wp-content/uploads/2024/01/neptune-badge.svg">
</a><a target="_blank" href="https://docs.neptune.ai/integrations/tensorflow/">
  <img alt="View tutorial in docs" src="https://neptune.ai/wp-content/uploads/2024/01/docs-badge-2.svg">
</a>

In this example, we will use Neptune to log metadata generated from training using TensorFlow.

By the end of this guide, you will be able to
* Track and version the data.
* Log losses and other metrics generated from training.
* Log prediction over multiple epochs.
* Save the generated model with model registry.

## Before you start

This notebook example lets you try out Neptune anonymously, with zero setup.

If you want to see the example logged to your own workspace instead:

  1. Create a Neptune account. [Register &rarr;](https://neptune.ai/register)
  1. Create a Neptune project that you will use for tracking metadata. For instructions, see [Creating a project](https://docs.neptune.ai/setup/creating_project) in the Neptune docs.

## Install Neptune and dependencies

In [ ]:
! pip install -U neptune tensorflow numpy requests

## Start a run

To create a new run for tracking the metadata, you tell Neptune who you are (`api_token`) and where to send the data (`project`).

You can use the default code cell below to create an anonymous run in the public project [common/tensorflow-support](https://app.neptune.ai/common/tensorflow-support). **Note**: Public projects are cleaned regularly, so anonymous runs are only stored temporarily.

### Log to your own project instead

Replace the code below with the following:

```python
import neptune
from getpass import getpass

run = neptune.init_run(
    project="workspace-name/project-name",  # replace with your own (see instructions below)
    api_token=getpass("Enter your Neptune API token: "),
)
```

To find your API token and full project name:

1. [Log in to Neptune](https://app.neptune.ai/).
1. In the bottom-left corner, expand your user menu and select **Get your API token**.
1. The workspace name is displayed in the top-left corner of the app. To copy the project path, in the top-right corner, open the settings menu and select **Properties**.

For more help, see [Setting Neptune credentials](https://docs.neptune.ai/setup/setting_credentials) in the Neptune docs.

In [ ]:
import neptune

run = neptune.init_run(
    api_token=neptune.ANONYMOUS_API_TOKEN,
    project="common/tensorflow-support",
)

**To open the run in the Neptune web app, click the link that appeared in the cell output.**

We'll use the `run` object we just created to log metadata. You'll see the metadata appear in the app.

## Log metadata to Neptune



Import libraries

In [ ]:
import io

import requests
import tensorflow as tf
import numpy as np

Download the MNIST Data

In [ ]:
response = requests.get("https://storage.googleapis.com/tensorflow/tf-keras-datasets/mnist.npz")
with open("mnist.npz", "wb") as f:
    f.write(response.content)

In [ ]:
# (Neptune) Track and version data files used for training
run["datasets/version"].track_files("mnist.npz")

In [ ]:
with np.load("mnist.npz") as data:
    train_examples = data["x_train"]
    train_labels = data["y_train"]
    test_examples = data["x_test"]
    test_labels = data["y_test"]

Parameters for training

In [ ]:
params = {
    "batch_size": 1024,
    "shuffle_buffer_size": 100,
    "lr": 0.001,
    "num_epochs": 10,
    "num_visualization_examples": 10,
}

In [ ]:
# (Neptune) Log training parameters
run["training/model/params"] = params

Normalize data for training

In [ ]:
def normalize_img(image):
    """Normalizes images: `uint8` -> `float32`."""
    return tf.cast(image, tf.float32) / 255.0


train_examples = normalize_img(train_examples)
test_examples = normalize_img(test_examples)

Prepare data for training

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_examples, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_examples, test_labels))

train_dataset = train_dataset.shuffle(params["shuffle_buffer_size"]).batch(params["batch_size"])
test_dataset = test_dataset.batch(params["batch_size"])

Prepare model

In [ ]:
# Model
model = tf.keras.models.Sequential(
    [
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(10),
    ]
)

# Loss
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Optimizer
optimizer = tf.keras.optimizers.Adam(params["lr"])

In [ ]:
with io.StringIO() as s:
    model.summary(print_fn=lambda x: s.write(x + "\n"))
    model_summary = s.getvalue()

# (Neptune) Log model summary
run["training/model/summary"] = model_summary

Helper functions for training loop

In [ ]:
def loss_and_preds(model, x, y, training):
    # training=training is needed only if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    y_ = model(x, training=training)

    return loss_object(y_true=y, y_pred=y_), y_


def grad(model, inputs, targets):
    with tf.GradientTape() as tape:
        loss_value, _ = loss_and_preds(model, inputs, targets, training=True)
    return loss_value, tape.gradient(loss_value, model.trainable_variables)

Training Loop

In [ ]:
for epoch in range(params["num_epochs"]):
    epoch_loss_avg = tf.keras.metrics.Mean()
    epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

    for x, y in train_dataset:
        loss_value, grads = grad(model, x, y)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        epoch_loss_avg.update_state(loss_value)
        epoch_accuracy.update_state(y, model(x, training=True))

    # (Neptune) Log metrics for the epoch
    # Train metrics
    run["training/train/loss"].append(epoch_loss_avg.result())
    run["training/train/accuracy"].append(epoch_accuracy.result())

    # (Neptune) Log test metrics
    test_loss, test_preds = loss_and_preds(model, test_examples, test_labels, False)
    run["training/test/loss"].append(test_loss)
    acc = epoch_accuracy(test_labels, test_preds)
    run["training/test/accuracy"].append(acc)

    # (Neptune) Log test prediction
    for idx in range(params["num_visualization_examples"]):
        np_image = test_examples[idx].numpy().reshape(28, 28)
        image = neptune.types.File.as_image(np_image)
        pred_label = test_preds[idx].numpy().argmax()
        true_label = test_labels[idx]
        run[f"training/visualization/epoch_{epoch}"].append(
            image, description=f"pred={pred_label} | actual={true_label}"
        )

    if epoch % 5 == 0 or epoch == (params["num_epochs"] - 1):
        print(
            "Epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3%}".format(
                epoch, epoch_loss_avg.result(), epoch_accuracy.result()
            )
        )

Tracking model with Neptune model registry

Refer to the [documentation](https://neptune.ai/product/model-registry) for more information.

In [ ]:
# (Neptune) Create a model_version object
model_version = neptune.init_model_version(
    model="TFSUP-TFMOD",
    project="common/tensorflow-support",
    api_token=neptune.ANONYMOUS_API_TOKEN,
)

In [ ]:
# (Neptune) Log metadata to model version
model_version["run_id"] = run["sys/id"].fetch()
model_version["metrics/test_loss"] = test_loss
model_version["metrics/test_accuracy"] = acc
model_version["datasets/version"].track_files("mnist.npz")

In [ ]:
# Saves model artifacts to "weights" folder
model.save("weights")

# (Neptune) Log model artifacts
model_version["model/weights"].upload_files("weights/*")

## Stop logging

Once you are done logging, stop tracking the run.

In [ ]:
run.stop()
model_version.stop()

## Explore the results in Neptune

You can also check out an [example run](https://app.neptune.ai/common/tensorflow-support/e/TFSUP-101) and the corresponding [model version](https://app.neptune.ai/common/tensorflow-support/m/TFSUP-TFMOD/v/TFSUP-TFMOD-112/metadata).